In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time

import src.utils.las_utils as las_utils
import src.utils.csv_utils as csv_utils
import src.utils.ahn_utils as ahn_utils
import src.fusion as fusion
from src.utils.labels import Labels
from src.pipeline import Pipeline

## Car fuser based on cluster dimensions and BGT information
First, download and parse the required BGT data in notebook [1. Generate reference data](1.%20Generate%20reference%20data.ipynb).

In [ ]:
# The already labelled ground and building points must be removed.
exclude_labels = (Labels.GROUND, Labels.BUILDING)

In [ ]:
# Select the file to process.
tilecode = '2386_9702'

in_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'

# Data folder for the fusers.
ahn_data_folder = '../datasets/ahn/'
pc_data_folder = '../datasets/pointcloud/'
bgt_data_file = '../datasets/bgt/bgt_roads_demo.csv'

In [ ]:
# Read NPZ file
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

# Car fuser using a clustering algorithm and BGT road data.
# To find appropriate dimension ranges, see e.g. https://nl.automobiledimension.com/.
car_fuser = fusion.CarFuser(Labels.CAR, ahn_reader, bgt_file=bgt_data_file,
                            grid_size=0.05, min_component_size=5000,
                            min_height=1.2, max_height=2.4,
                            min_width=1.4, max_width=2.4,
                            min_length=3.0, max_length=6.0)

## Process point clouds and save the results
The pipeline can now be used to process point clouds. Labels will be written to the LAS file as a new extended field called label.

In [ ]:
# Set-up the order in how we want to label.
process_sequence = (car_fuser,)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

## Process a single file

In [ ]:
# Process the file.
start = time.time()
pipeline.process_file(in_file, out_file=out_file)
end = time.time()
print(f'Tile labelled in {end-start:.2f} seconds.')